### Run t-route through BMI functions

Load libraries

In [1]:
import sys
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr

sys.path.append("../t-route/src/")
import bmi_troute

Load geopackage data tables

In [2]:
file_path = '/home/sean.horvath/projects/data/large_network/conus.gpkg'
flowpaths = gpd.read_file(file_path, layer='flowpaths')
flowpath_attributes = gpd.read_file(file_path, layer='flowpath_attributes')
lakes = gpd.read_file(file_path, layer='lakes')
network = gpd.read_file(file_path, layer='network')

Initialize our t-route model and set static variables

In [3]:
# Initialize model with configuration file
troute = bmi_troute.bmi_troute()
troute.initialize(bmi_cfg_file='/home/sean.horvath/projects/t-route/test/ngen/test_conus_AnA.yaml')


Provide t-route model each geopackage table column for static variables

In [4]:

troute.set_value('id', np.array(flowpaths['id']))
troute.set_value('toid', np.array(flowpaths['toid']))
troute.set_value('lengthkm', np.array(flowpaths['lengthkm']))
troute.set_value('attributes_id', np.array(flowpath_attributes['id']))
troute.set_value('rl_gages', np.array(flowpath_attributes['rl_gages']))
troute.set_value('rl_NHDWaterbodyComID', np.array(flowpath_attributes['rl_NHDWaterbodyComID']))
troute.set_value('MusK', np.array(flowpath_attributes['MusK']))
troute.set_value('MusX', np.array(flowpath_attributes['MusX']))
troute.set_value('n', np.array(flowpath_attributes['n']))
troute.set_value('So', np.array(flowpath_attributes['So']))
troute.set_value('ChSlp', np.array(flowpath_attributes['ChSlp']))
troute.set_value('BtmWdth', np.array(flowpath_attributes['BtmWdth']))
troute.set_value('nCC', np.array(flowpath_attributes['nCC']))
troute.set_value('TopWdthCC', np.array(flowpath_attributes['TopWdthCC']))
troute.set_value('TopWdth', np.array(flowpath_attributes['TopWdth']))
troute.set_value('hl_link', np.array(lakes['hl_link']))
troute.set_value('ifd', np.array(lakes['ifd']))
troute.set_value('LkArea', np.array(lakes['LkArea']))
troute.set_value('LkMxE', np.array(lakes['LkMxE']))
troute.set_value('OrificeA', np.array(lakes['OrificeA']))
troute.set_value('OrificeC', np.array(lakes['OrificeC']))
troute.set_value('OrificeE', np.array(lakes['OrificeE']))
troute.set_value('WeirC', np.array(lakes['WeirC']))
troute.set_value('WeirE', np.array(lakes['WeirE']))
troute.set_value('WeirL', np.array(lakes['WeirL']))
troute.set_value('network_id', np.array(network['id']))
troute.set_value('hydroseq', np.array(network['hydroseq']))
troute.set_value('hl_uri', np.array(network['hl_uri']))


Read forcing files

In [5]:
forcing_files = glob.glob('/home/sean.horvath/projects/data/large_network/sample_qlat_files/2021*')
forcing_vals = pd.DataFrame()
for file in forcing_files:
    df = pd.read_csv(file).set_index('feature_id')
    if forcing_vals.empty:
        forcing_vals = df
    else:
        forcing_vals = pd.merge(forcing_vals, df, on='feature_id')

Read DA data

In [37]:
usgs_timeslice_files = glob.glob('/home/sean.horvath/projects/data/extended_AnA/usgs_timeslices/*')
usgs_timeslice = pd.DataFrame()
for file in usgs_timeslice_files:
    df = xr.open_dataset(file).to_dataframe()
    usgs_timeslice = pd.concat([usgs_timeslice, df])

In [35]:
usace_timeslice_files = glob.glob('/home/sean.horvath/projects/data/extended_AnA/usace_timeslices/*')
usace_timeslice = pd.DataFrame()
for file in usace_timeslice_files:
    df = xr.open_dataset(file).to_dataframe()
    if usace_timeslice.empty:
        usace_timeslice = df
    else:
        usace_timeslice = pd.concat([usace_timeslice, df])

Run our model for 1 hour

In [7]:

# Dataframes for saving all output
full_fvd = pd.DataFrame()

# Loop through hourly timesteps calling update_until
for hr in range(4):
    # Full network
    # Set dynamic values
    troute.set_value('land_surface_water_source__volume_flow_rate', np.array(forcing_vals.iloc[:,hr]))
    troute.set_value('land_surface_water_source__id', np.array(forcing_vals.index))

    # Set duration to run model for
    nts = 12
    n = nts*300

    # Run our model
    troute.update_until(n)


    


/home/sean.horvath/projects/t-route/src/troute-routing/troute/routing/compute.py:520: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pd.Series(index=lastobs_df_sub.index, name="Null"),
/home/sean.horvath/projects/t-route/src/troute-routing/troute/routing/compute.py:524: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pd.Series(index=lastobs_df_sub.index, name="Null"),
/home/sean.horvath/projects/t-route/src/troute-routing/troute/routing/compute.py:520: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pd.Series(index=lastobs_df_sub.index, name="Null"),
/home/sean.horvath/projects/t-route/src/troute-routing/troute/routing/compute.py:524: FutureWarni

/home/sean.horvath/projects/t-route/src/troute-routing/troute/routing/compute.py:520: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pd.Series(index=lastobs_df_sub.index, name="Null"),
/home/sean.horvath/projects/t-route/src/troute-routing/troute/routing/compute.py:524: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pd.Series(index=lastobs_df_sub.index, name="Null"),
/home/sean.horvath/projects/t-route/src/troute-routing/troute/routing/compute.py:520: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pd.Series(index=lastobs_df_sub.index, name="Null"),
/home/sean.horvath/projects/t-route/src/troute-routing/troute/routing/compute.py:524: FutureWarni

/home/sean.horvath/projects/t-route/src/troute-routing/troute/routing/compute.py:520: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pd.Series(index=lastobs_df_sub.index, name="Null"),
/home/sean.horvath/projects/t-route/src/troute-routing/troute/routing/compute.py:524: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pd.Series(index=lastobs_df_sub.index, name="Null"),
/home/sean.horvath/projects/t-route/src/troute-routing/troute/routing/compute.py:520: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pd.Series(index=lastobs_df_sub.index, name="Null"),
/home/sean.horvath/projects/t-route/src/troute-routing/troute/routing/compute.py:524: FutureWarni

/home/sean.horvath/projects/t-route/src/troute-routing/troute/routing/compute.py:520: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pd.Series(index=lastobs_df_sub.index, name="Null"),
/home/sean.horvath/projects/t-route/src/troute-routing/troute/routing/compute.py:524: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pd.Series(index=lastobs_df_sub.index, name="Null"),
/home/sean.horvath/projects/t-route/src/troute-routing/troute/routing/compute.py:520: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pd.Series(index=lastobs_df_sub.index, name="Null"),
/home/sean.horvath/projects/t-route/src/troute-routing/troute/routing/compute.py:524: FutureWarni